<a href="https://colab.research.google.com/github/Tonic4to/Disease-Identification-And-prevention/blob/main/Pest_%2B_Disease_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Models:

CNN image classification - Identifies the crop, identifies the parasite.

# CNN Image Classification


In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from google.colab import auth
from google.colab import drive
import json
import os

In [3]:
# Authenticate and get Google Drive access
auth.authenticate_user()

# Use the secrets stored in Colab
# To access secrets, use the `os.environ.get` method
token = os.getenv('GITHUB_TOKEN')


In [4]:
#Depreciated
'''
# Authenticate and mount Google Drive.
auth.authenticate_user()
drive.mount('/content/drive')

# Path to the file where the token is stored (e.g., /content/drive/My Drive/colab_secrets.json)
secrets_file = '/content/drive/My Drive/colab_secrets.json'
'
# Load the token from the secrets file
with open(secrets_file) as f:
    secrets = json.load(f)
    token = secrets['GITHUB_TOKEN']
'''

#Depreciated
'''
# Authenticate and get Google Drive access
auth.authenticate_user()

secrets = output.eval_js('google.colab.kernel.getSecret("GITHUB_TOKEN")')
token = secrets['GITHUB_TOKEN']
'''
# Install Kaggle package
!pip install kaggle

# Install dos2unix
!apt-get install dos2unix

# Remove the existing repository directory if it exists
!rm -rf Disease-Identification-And-prevention

# Clone the repository using the token
!git clone https://{token}@github.com/Tonic4to/Disease-Identification-And-prevention.git

# Convert the line endings of the shell script
!dos2unix Disease-Identification-And-prevention/config/setup_kaggle.sh

# Run the shell script to set up Kaggle configuration
!bash Disease-Identification-And-prevention/config/setup_kaggle.sh

# Download a dataset using the Kaggle API
!kaggle datasets download -d marquis03/plants-classification

# Create a directory to unzip the dataset
!mkdir -p dataset_directory

# Unzip the dataset and overwrite files without prompting
!unzip -o plants-classification.zip -d dataset_directory

# List the contents of the dataset directory to verify
!ls dataset_directory


Streaming output truncated to the last 5000 lines.
  inflating: dataset_directory/train/tobacco/tobacco191.jpg  
  inflating: dataset_directory/train/tobacco/tobacco192.jpg  
  inflating: dataset_directory/train/tobacco/tobacco193.jpg  
  inflating: dataset_directory/train/tobacco/tobacco194.jpg  
  inflating: dataset_directory/train/tobacco/tobacco195.jpg  
  inflating: dataset_directory/train/tobacco/tobacco196.jpg  
  inflating: dataset_directory/train/tobacco/tobacco197.jpg  
  inflating: dataset_directory/train/tobacco/tobacco198.jpg  
  inflating: dataset_directory/train/tobacco/tobacco199.jpg  
  inflating: dataset_directory/train/tobacco/tobacco2.jpg  
  inflating: dataset_directory/train/tobacco/tobacco20.jpg  
  inflating: dataset_directory/train/tobacco/tobacco200.jpg  
  inflating: dataset_directory/train/tobacco/tobacco201.jpg  
  inflating: dataset_directory/train/tobacco/tobacco202.jpg  
  inflating: dataset_directory/train/tobacco/tobacco203.jpg  
  inflating: dataset_d

In [ ]:
'''
#Depreciated - use bash script method
!git clone https://github.com/Tonic4to/Disease-Identification-And-prevention.git
!pip install kaggle
%cd Disease-Identification-And-prevention/config
#Moves Kaggle.json to API location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#Downloads Dataset
!kaggle datasets download -d marquis03/plants-classification
'''

In [25]:

#Rename
# Load the CSV files
train_labels = pd.read_csv('/content/dataset_directory/train.csv')
test_labels = pd.read_csv('/content/dataset_directory/test.csv')
val_labels = pd.read_csv('/content/dataset_directory/val.csv')

# Display the first few rows of each DataFrame
print("Train Labels:")
print(train_labels.head())

print("\nTest Labels:")
print(test_labels.head())

print("\nValidation Labels:")
print(val_labels.head())


Train Labels:
                     image:FILE  category
0  train/aloevera/aloevera0.jpg         0
1  train/aloevera/aloevera1.jpg         0
2  train/aloevera/aloevera2.jpg         0
3  train/aloevera/aloevera3.jpg         0
4  train/aloevera/aloevera4.jpg         0

Test Labels:
                      image:FILE  category
0  test/aloevera/aloevera800.jpg         0
1  test/aloevera/aloevera801.jpg         0
2  test/aloevera/aloevera802.jpg         0
3  test/aloevera/aloevera803.jpg         0
4  test/aloevera/aloevera804.jpg         0

Validation Labels:
                     image:FILE  category
0  val/aloevera/aloevera700.jpg         0
1  val/aloevera/aloevera701.jpg         0
2  val/aloevera/aloevera702.jpg         0
3  val/aloevera/aloevera703.jpg         0
4  val/aloevera/aloevera704.jpg         0


In [26]:
#Verifying Directory Contents
# List the contents of the dataset directory
print("Contents of dataset_directory:")
print(os.listdir('/content/dataset_directory'))

# If the CSV files are in subdirectories, list their contents
print("\nContents of train directory:")
print(os.listdir('/content/dataset_directory/train'))

print("\nContents of test directory:")
print(os.listdir('/content/dataset_directory/test'))

print("\nContents of val directory:")
print(os.listdir('/content/dataset_directory/val'))


Contents of dataset_directory:
['train', 'val', 'val.csv', 'test', 'train.csv', 'test.csv']

Contents of train directory:
['bilimbi', 'curcuma', 'waterapple', 'cassava', 'banana', 'cucumber', 'coconut', 'soybeans', 'guava', 'mango', 'papaya', 'tobacco', 'longbeans', 'pomelo', 'spinach', 'kale', 'melon', 'paddy', 'orange', 'ginger', 'pineapple', 'cantaloupe', 'peperchili', 'sweetpotatoes', 'watermelon', 'aloevera', 'galangal', 'classname.txt', 'eggplant', 'corn', 'shallot']

Contents of test directory:
['bilimbi', 'curcuma', 'waterapple', 'cassava', 'banana', 'cucumber', 'coconut', 'soybeans', 'guava', 'mango', 'papaya', 'tobacco', 'longbeans', 'pomelo', 'spinach', 'kale', 'melon', 'paddy', 'orange', 'ginger', 'pineapple', 'cantaloupe', 'peperchili', 'sweetpotatoes', 'watermelon', 'aloevera', 'galangal', 'classname.txt', 'eggplant', 'corn', 'shallot']

Contents of val directory:
['bilimbi', 'curcuma', 'waterapple', 'cassava', 'banana', 'cucumber', 'coconut', 'soybeans', 'guava', 'mango'

In [47]:
#Helper Function Time
# Define image size
IMG_SIZE = (128, 128)

def load_and_process_images(file_path):
  #load image (Might take a while because we are loading a lot)
  image = load_img(file_path, target_size = IMG_SIZE)
  #convert to array
  image = img_to_array(image)
  #Normalize
  image = image /255.0
  return image
'''
def load_dataset(csv_file, base_dir, batch_size=32):
    def gen():
        for _, row in csv_file.iterrows():
            file_path = os.path.join(base_dir, row['image:FILE'])
            image = load_and_process_images(file_path)
            label = row['category']
            yield image, label
'''
def load_dataset(csv_file, base_dir, batch_size=32):
    def gen():
        for _, row in csv_file.iterrows():
            # Adjust this if needed to match the exact path structure
            file_path = os.path.join(base_dir, row['image:FILE'])
            if not os.path.exists(file_path):
                print(f"Warning: File not found {file_path}")
                continue  # Skip missing files
            image = load_and_process_images(file_path)
            label = row['category']
            yield image, label

    dataset = tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )
    dataset = dataset.batch(batch_size)
    return dataset


# Load datasets
batch_size = 32
train_dataset = load_dataset(train_labels, '/content/dataset_directory/train', batch_size)
test_dataset = load_dataset(test_labels, '/content/dataset_directory/test', batch_size)
val_dataset = load_dataset(val_labels, '/content/dataset_directory/val', batch_size)

'''
#Depreciated
#Validate shapes
print(f"Train images shape: {train_images.shape}")
print(f"Test images shape: {test_images.shape}")
print(f"Validation images shape: {val_images.shape}")
'''

'\n#Depreciated\n#Validate shapes\nprint(f"Train images shape: {train_images.shape}")\nprint(f"Test images shape: {test_images.shape}")\nprint(f"Validation images shape: {val_images.shape}")\n'

In [48]:
# Inspect a sample from the dataset
for image_batch, label_batch in train_dataset.take(1):
    print("Sample image batch shape:", image_batch.shape)
    print("Sample label batch shape:", label_batch.shape)
    print("Sample labels:", label_batch.numpy())


Streaming output truncated to the last 5000 lines.


In [41]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [42]:
#Create a model


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_8 (Flatten)         (None, 57600)             0         
                                                                 
 dense_16 (Dense)            (None, 128)              

In [43]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [44]:
def print_directory_contents(path):
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

print_directory_contents('/content/dataset_directory')

Streaming output truncated to the last 5000 lines.
            cucumber964.jpg
            cucumber910.jpg
            cucumber869.jpg
            cucumber891.jpg
            cucumber865.jpg
            cucumber939.jpg
            cucumber921.jpg
            cucumber972.jpg
            cucumber983.jpg
            cucumber931.jpg
            cucumber943.jpg
            cucumber985.jpg
            cucumber945.jpg
            cucumber987.jpg
            cucumber825.jpg
            cucumber956.jpg
            cucumber920.jpg
            cucumber962.jpg
            cucumber954.jpg
            cucumber905.jpg
            cucumber827.jpg
            cucumber860.jpg
            cucumber922.jpg
            cucumber970.jpg
            cucumber813.jpg
            cucumber961.jpg
            cucumber876.jpg
            cucumber808.jpg
            cucumber928.jpg
            cucumber957.jpg
            cucumber811.jpg
            cucumber880.jpg
            cucumber932.jpg
            cucumber918.j

In [45]:
# Train the model using the dataset
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=10)

Streaming output truncated to the last 5000 lines.


ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.